In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, cross_val_score

ModuleNotFoundError: No module named 'pandas'

In [ ]:
# import dataset
df = pd.read_csv('/content/train_diabetes_health_indicators.csv', delimiter=',')
df.dataframeName = 'diabetes_health.csv'
nRow, nCol = df.shape
print(f'There are {nRow} rows and {nCol} columns')

In [ ]:
# valeur manquant
print(f'Colonnes avec valeurs manquantes : { df.isnull().sum()} .')

In [ ]:
df.describe

In [ ]:
df= df.drop("Unnamed: 0", axis=1)

In [ ]:
# les instances dupliquer
lignes_dupliquees = df.duplicated()
print(df[lignes_dupliquees])

#supprision donnees duplique
nombre_lignes_dupliquees = df.duplicated().sum()
print(f"Nombre de lignes dupliquées : {nombre_lignes_dupliquees}")

df = df.drop_duplicates()

In [ ]:
lignes_dupliquees = df.duplicated()
print(df[lignes_dupliquees])


In [ ]:
print(f'Les valeurs que prend la cible sont {df["Diabetes_012"].unique()} ')
diabetes_target = df["Diabetes_012"]

In [ ]:
counts = diabetes_target.value_counts()

plt.figure(figsize=(8, 8))
plt.pie(counts, labels=counts.index, autopct='%1.1f%%', startangle=90, colors=['#66b3ff', '#99ff99', '#ffcc99'])
plt.title('Proportion des Classes')
plt.show()

In [ ]:
colonnes_catégorielles = df.select_dtypes(include=['object']).columns

#print("Colonnes catégorielles :")
print(colonnes_catégorielles)

In [ ]:
from scipy.stats import zscore

df_grouped = df.groupby(["Diabetes_012"]).mean().apply(lambda col: zscore(col))

df_grouped.T.plot.bar()
plt.show()

df_0 = df[df["Diabetes_012"] == 0]
df_1 = df[df["Diabetes_012"] == 1]
df_2 = df[df["Diabetes_012"] == 2]

X_0 = df_0.drop("Diabetes_012", axis=1)
X_1 = df_1.drop("Diabetes_012", axis=1)
X_2 = df_2.drop("Diabetes_012", axis=1)

X_0_normalized = X_0.apply(lambda col: zscore(col))
X_1_normalized = X_1.apply(lambda col: zscore(col))
X_2_normalized = X_2.apply(lambda col: zscore(col))



In [ ]:
for col in df.columns[1:]:  # Excluez la colonne 'Target'
    plt.figure(figsize=(8, 4))
    plt.boxplot([X_0[col], X_1[col]])
    plt.title(f'Histogramme de {col} pour les classes 0 et 1')
    plt.xlabel(col)
    plt.ylabel('Fréquence')
    plt.legend(loc='upper right')
    plt.show()

In [ ]:
import seaborn as sns

correlation_matrix = df.corr()

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=False, cmap='coolwarm', fmt=".1f", linewidths=0.5)
plt.title('Matrice de Corrélation')
plt.show()

In [ ]:
# pour verifier si on peut supprimer les variable qui ont correlation plus que 0,8

import pandas as pd
import numpy as np

# Supposons que vous ayez déjà défini df

# Calculer la matrice de corrélation
correlation_matrix = df.corr().abs()

# Sélectionner la partie supérieure de la matrice de corrélation
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(np.bool_))

# Trouver les colonnes corrélées avec un seuil
threshold = 0.8
correlated_columns = [column for column in upper_triangle.columns if any(upper_triangle[column] > threshold)]

# Supprimer les colonnes corrélées
df_no_corr = df.drop(columns=correlated_columns)


In [ ]:

scaler = StandardScaler()
# Apply standardization to the entire DataFrame
df_standardized = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [ ]:
df = df.drop("Diabetes_012", axis=1)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df_standardized, diabetes_target, stratify=diabetes_target, random_state=42)

In [ ]:
# Créez des poids d'échantillonnage en fonction des classes
class_weights = {0: 1, 1: 2, 2: 1}  # Ajustez les poids selon vos besoins

# Créez un tableau de poids pour chaque exemple d'entraînement
sample_weights = [class_weights[label] for label in y_train]

In [ ]:
# utilisation de cross validation
from sklearn.model_selection import StratifiedKFold, cross_val_score
# Utilisez la validation croisée stratifiée avec 5 plis
stratified_kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


In [ ]:
from sklearn.svm import SVC

# Créez un modèle SVM
svm_model = SVC(class_weight=class_weights, random_state=42)

# Entraînez le modèle sur les données d'entraînement
svm_model.fit(X_train, y_train, sample_weight=sample_weights)

# Prédisez les classes sur les données de test
y_pred = svm_model.predict(X_test)
# Mesurez les performances du modèle
accuracy = metrics.accuracy_score(y_test, y_pred)
print(accuracy)




In [ ]:
import pickle
pickle.dump(svm_model, open("SVM.pkl", "wb"))